In [1]:
from numpy.core.numeric import tensordot
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import time
import mediapipe as mp
import math
import csv
import plotly.offline as po
#import plotly.graph_objs as go
import plotly.graph_objects as go
import pandas as pd

po.init_notebook_mode(connected=True)

# def F(a,b):
#     scatt = fig.add_scatter3d(
#         x = marker_x,
#         y = marker_y,
#         z = marker_z,
#         mode = 'marker',
#         marker = dict(
#             color = 'rgb(100,100,200)',
#             size = 3,
#             opacity = 0.8
#         ))
            
#     data.append(go.Scatter3d(
#         x = [true_x[a], true_x[b]],
#         y = [true_y[a], true_y[b]],
#         z = [true_z[a], true_z[b]],
#             mode = 'lines',
#             marker = dict(
#             color = 'rgb(100,100,200)',
#             size = 3,
#             opacity = 0.8
#         )
#     ))

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# make dir
timestr = time.strftime("%y%m%d-%H%M%S")
dirname = "recordimages/%s" % timestr
os.makedirs(dirname)


# ストリーム(Depth/Color)の設定
w = 1280
h = 720
config = rs.config()
config.enable_stream(rs.stream.color, w, h, rs.format.bgr8, 15)
config.enable_stream(rs.stream.depth, w, h, rs.format.z16, 15)

# ストリーミング開始
pipeline = rs.pipeline()
profile = pipeline.start(config)

# Alignオブジェクト生成
align_to = rs.stream.color
align = rs.align(align_to)

try:
    count = 0

    with open('%s.csv' % timestr, 'w', newline="") as f:
        writer = csv.writer(f)

        # CSVファイルの1行目を書き込み
        csv_header = ["FrameNo.", "label", "label_count", "1_x", "1_x_est", "1_x_err", "1_y", "1_y_est", "1_y_err", "1_z", "1_z_est", "1_z_err", "2_x", "2_x_est", "2_x_err", "2_y", "2_y_est", "2_y_err", "2_z", "2_z_est", "2_z_err", "upperarm01.L_x", "upperarm01.L_x_est", "upperarm01.L_x_err", "upperarm01.L_y", "upperarm01.L_y_est", "upperarm01.L_y_err", "upperarm01.L_z", "upperarm01.L_z_est", "upperarm01.L_z_err", "upperarm01.R_x", "upperarm01.R_x_est", "upperarm01.R_x_err", "upperarm01.R_y", "upperarm01.R_y_est", "upperarm01.R_y_err", "upperarm01.R_z", "upperarm01.R_z_est", "upperarm01.R_z_err", "LELB_x", "LELB_x_est", "LELB_x_err", "LELB_y", "LELB_y_est", "LELB_y_err", "LELB_z", "LELB_z_est", "LELB_z_err", "LIEL_x", "LIEL_x_est", "LIEL_x_err", "LIEL_y", "LIEL_y_est", "LIEL_y_err", "LIEL_z", "LIEL_z_est", "LIEL_z_err", "RELB_x", "RELB_x_est", "RELB_x_err", "RELB_y", "RELB_y_est", "RELB_y_err", "RELB_z", "RELB_z_est", "RELB_z_err", "RIEL_x", "RIEL_x_est", "RIEL_x_err", "RIEL_y", "RIEL_y_est", "RIEL_y_err", "RIEL_z", "RIEL_z_est", "RIEL_z_err", "LIWR_x", "LIWR_x_est", "LIWR_x_err", "LIWR_y", "LIWR_y_est", "LIWR_y_err", "LIWR_z", "LIWR_z_est", "LIWR_z_err", "LOWR_x", "LOWR_x_est", "LOWR_x_err", "LOWR_y", "LOWR_y_est", "LOWR_y_err", "LOWR_z", "LOWR_z_est", "LOWR_z_err", "_x", "_x_est", "_x_err", "_y", "_y_est", "_y_err", "_z", "_z_est",
                      "_z_err", "RIWR_x", "RIWR_x_est", "RIWR_x_err", "RIWR_y", "RIWR_y_est", "RIWR_y_err", "RIWR_z", "RIWR_z_est", "RIWR_z_err", "ROWR_x", "ROWR_x_est", "ROWR_x_err", "ROWR_y", "ROWR_y_est", "ROWR_y_err", "ROWR_z", "ROWR_z_est", "ROWR_z_err", "E_x", "E_x_est", "E_x_err", "E_y", "E_y_est", "E_y_err", "E_z", "E_z_est", "E_z_err", "5_x", "5_x_est", "5_x_err", "5_y", "5_y_est", "5_y_err", "5_z", "5_z_est", "5_z_err", "6_x", "6_x_est", "6_x_err", "6_y", "6_y_est", "6_y_err", "6_z", "6_z_est", "6_z_err", "7_x", "7_x_est", "7_x_err", "7_y", "7_y_est", "7_y_err", "7_z", "7_z_est", "7_z_err", "8_x", "8_x_est", "8_x_err", "8_y", "8_y_est", "8_y_err", "8_z", "8_z_est", "8_z_err", "9_x", "9_x_est", "9_x_err", "9_y", "9_y_est", "9_y_err", "9_z", "9_z_est", "9_z_err", "10_x", "10_x_est", "10_x_err", "10_y", "10_y_est", "10_y_err", "10_z", "10_z_est", "10_z_err", "11_x", "11_x_est", "11_x_err", "11_y", "11_y_est", "11_y_err", "11_z", "11_z_est", "11_z_err", "_x", "_x_est", "_x_err", "_y", "_y_est", "_y_err", "_z", "_z_est", "_z_err", "LHEE_x", "LHEE_x_est", "LHEE_x_err", "LHEE_y", "LHEE_y_est", "LHEE_y_err", "LHEE_z", "LHEE_z_est", "LHEE_z_err", "E_x", "E_x_est", "E_x_err", "E_y", "E_y_est", "E_y_err", "E_z", "E_z_est", "E_z_err", "RHEE_x", "RHEE_x_est", "RHEE_x_err", "RHEE_y", "RHEE_y_est", "RHEE_y_err", "RHEE_z", "RHEE_z_est", "RHEE_z_err"]
        writer.writerow(csv_header)
            #カメラ位置
        camera = dict(
            eye = dict(x = 3.5, y = 0.05, z = 0.05 )
        )

        # traceを作成
        data = []
        marker_x = [1, 0, 0]
        marker_y = [0, 2, 0]
        marker_z = [0, 0, 3]

        fig = go.FigureWidget()

        # マーカー
        scatt = fig.add_scatter3d(
            x = marker_x,
            y = marker_y,
            z = marker_z,
            mode = 'markers',
            marker = dict(
                color = 'rgb(100,100,200)',
                size = 3,
                opacity = 0.8
            ))
        
        # traceとlayoutからfigureを作成
        fig.update_layout(
            height = 500,
            width = 500,
            margin = dict(
                l = 0,
                r = 0,
                b = 0,
                t = 0
            ),
            #カメラ位置
            scene_camera = camera,
            scene = dict(
                aspectmode = 'cube',
                #aspectratio = dict(x=1, y=1, z=1),
                xaxis = dict(range = [-1.0, 1.0]),
                yaxis = dict(range = [-1.0 ,1.0 ]),
                zaxis = dict(range = [0, 3])),
            
            #scene=dict(aspectmode='cube'),
            showlegend=False,
            )
        display(fig)

        while True:
            flag = True
            XYZlist = []

            # フレーム待ち(Color & Depth)
            frames = pipeline.wait_for_frames()

            aligned_frames = align.process(frames)
            color_frame = aligned_frames.get_color_frame()
            depth_frame = aligned_frames.get_depth_frame()
            
            if not depth_frame or not color_frame:
                continue

            # imageをnumpy arrayに
            color_image = np.asanyarray(color_frame.get_data())
            depth_image = np.asanyarray(depth_frame.get_data())

            # test.pyより
            with mp_pose.Pose(
                    min_detection_confidence = 0.5,
                    min_tracking_confidence = 0.5) as pose:

                # image = cv2.cvtColor(cv2.flip(color_image, 1), cv2.COLOR_BGR2RGB)
                image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                image.flags.writeable = False
                results = pose.process(image)

                # Draw the pose annotation on the image.
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                # cv2.imshow('MediaPipe Pose', image)
                # if cv2.waitKey(5) & 0xFF == 27:
                #     break

                # pose_landmarksが取得されていなければcontinueで継続
                if not results.pose_landmarks:
                    continue

                # 取得されていればランドマークの座標を出力
                # 例として鼻（nose）の値を出力、fはフォーマット文字列で、書式を整えて出力できる
                # フォーマット文字列に変数の値を埋め込む場合は、{ }で囲んで埋め込む
                # distは辞書型、x, yはmediapipeから取得、それをもとにZ（深度）はRealSence2から取得
                # dist = {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                #      "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y}
                # print(dist["x"],dist["y"])

                # FocalLength(X,Y,Z)
                fx = 1280 / (2 * math.tan(math.radians(69.4) / 2))
                fy = 720 / (2 * math.tan(math.radians(42.5) / 2))

                # for lmk in results.pose_landmarks.landmark:
                lmks = [results.pose_landmarks.landmark[23],
                        results.pose_landmarks.landmark[24],
                        results.pose_landmarks.landmark[25],
                        results.pose_landmarks.landmark[26],
                        results.pose_landmarks.landmark[27],
                        results.pose_landmarks.landmark[28],
                        results.pose_landmarks.landmark[29],
                        results.pose_landmarks.landmark[30],
                        results.pose_landmarks.landmark[31],
                        results.pose_landmarks.landmark[32]]

                # CSVファイルの2行目の準備
                XYZlist = [count, "label", count % 901,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 0, 0]

                # 設定したランドマークについて繰り返し
                for lmk in lmks:
                    dist = {"x": lmk.x, "y": lmk.y}
                    try:
                        dist["Z"] = depth_frame.get_distance(
                            int(dist["x"]) * w, int(dist["y"]) * h)
                    except:
                        # 異常値が検出された場合はフラグを下げる
                        flag = False
                        break

                    X = dist["Z"] / fx * (dist["x"] * w - w / 2)
                    Y = dist["Z"] / fy * (dist["y"] * h - h / 2)
                    Z = dist["Z"]

                    #X->Z Y->X Z->Y
                    XYZlist.extend([Z, 0, 0, X, 0, 0, Y, 0, 0])
                    
                    # マーカーリストに追加する
                    marker_x.append(X)
                    marker_y.append(Y)
                    marker_z.append(Z)

                # 異常値が検出された場合は以降の処理は行わない
                if flag == False:
                    continue

                # save (X, Y, Z) to CSV
                writer.writerow(XYZlist)
                
            # グラフを再描画する
            with fig.batch_update():
                scatt.data[0].x = marker_x
                scatt.data[0].y = marker_y
                scatt.data[0].z = marker_z
                        
            # マーカーリストを初期化しておく
            marker_x = []
            marker_y = []
            marker_z = []
            
            # save images
            colorfilepath = dirname + "/color%5d.png" % count
            depthfilepath = dirname + "/depth%5d.png" % count
            mpfilepath = dirname + "/mp%5d.png" % count
            cv2.imwrite(colorfilepath, color_image)
            cv2.imwrite(depthfilepath, depth_image)
            cv2.imwrite(mpfilepath, image)

            # depth imageをカラーマップに変換
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(
                depth_image, alpha = 0.08), cv2.COLORMAP_JET)

            # 画像表示
            color_image_s = cv2.resize(color_image, (640, 360))
            depth_colormap_s = cv2.resize(depth_colormap, (640, 360))
            mp_image_s = cv2.resize(image, (640, 360))

            images_h1 = np.hstack((color_image_s, depth_colormap_s))
            images_h2 = np.hstack((mp_image_s, mp_image_s))
            images_v = np.vstack((images_h1, images_h2))
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RealSense', images_v)

            if cv2.waitKey(1) & 0xff == 27:  # ESCで終了
                cv2.destroyAllWindows()
                break

            count += 1

finally:

    # ストリーミング停止
    pipeline.stop()
    print("end")

FigureWidget({
    'data': [{'marker': {'color': 'rgb(100,100,200)', 'opacity': 0.8, 'size': 3},
             …

end


KeyboardInterrupt: 